# ⚙️ Python Generators: Memory-Efficient Iteration and Data Pipelines

**Welcome!** This notebook explores Python Generators, a powerful and memory-efficient way to create iterators. We'll delve into how they work using the `yield` keyword, contrast them with regular functions and list comprehensions, and see how they enable lazy evaluation for handling large datasets and building elegant data processing pipelines.

**Target Audience:** Python developers looking to understand iterators, generators, and lazy evaluation for writing more efficient and Pythonic code.

**Learning Objectives:**
*   Understand Python's iteration protocol (`__iter__`, `__next__`, `StopIteration`).
*   Learn how generator functions use `yield` to create iterators.
*   Understand the concept of lazy evaluation and its memory benefits.
*   Create concise generators using generator expressions.
*   Learn how to chain generators to build data pipelines.
*   Use `yield from` to delegate iteration to sub-generators.
*   Briefly touch upon asynchronous generators.
*   Identify best practices and common use cases for generators.

## 1. Introduction: The Problem with Large Sequences

Imagine you need to process a file containing millions of log entries, or generate a sequence of the first billion Fibonacci numbers. Storing such massive sequences entirely in memory using lists can be inefficient or even impossible due to memory constraints.

**Generators provide a solution:** They allow you to define iterable sequences where items are generated **one at a time** and **only when requested**. This is known as **lazy evaluation**.

**Analogy: The Streaming Service vs. Downloading Everything**

*   **Lists/Tuples:** Like downloading an entire movie series Box Set before you can watch the first episode. It takes up a lot of disk space (memory) upfront.
*   **Generators:** Like a streaming service (Netflix, YouTube). You request the next chunk of video (the next item) only when you're ready to watch it. The service generates and sends just that chunk, without needing to send the entire series at once. It's efficient for large content and you can even handle potentially infinite streams (like a live broadcast).

Generators are a fundamental concept for writing memory-efficient and elegant data processing code in Python.

## 2. The Foundation: Iterables and Iterators

To understand generators, we first need to understand Python's iteration protocol.

*   **Iterable:** An object capable of returning its members one at a time. Examples include lists, tuples, dictionaries, sets, strings, files, and generators. An object is iterable if it implements the `__iter__()` method, which must return an iterator.
*   **Iterator:** An object representing a stream of data. It must implement the iterator protocol, consisting of:
    *   `__iter__()`: Returns the iterator object itself.
    *   `__next__()`: Returns the next item from the stream. When there are no more items, it raises the `StopIteration` exception.

Python's `for` loop automatically handles this protocol: it calls `iter()` on the iterable to get an iterator, and then repeatedly calls `next()` on the iterator until `StopIteration` is caught.

In [1]:
my_list = ['a', 'b', 'c']
print(f"Is list iterable? {'__iter__' in dir(my_list)}")

# Get an iterator from the list
my_iterator = iter(my_list)
print(f"Type of iterator: {type(my_iterator)}")
print(f"Does iterator have __iter__? {'__iter__' in dir(my_iterator)}")
print(f"Does iterator have __next__? {'__next__' in dir(my_iterator)}")

# Manually iterating
print("\nManual iteration:")
try:
    print(f"  Item 1: {next(my_iterator)}")
    print(f"  Item 2: {next(my_iterator)}")
    print(f"  Item 3: {next(my_iterator)}")
    print(f"  Item 4: {next(my_iterator)}") # This will raise StopIteration
except StopIteration:
    print("  Caught StopIteration - iterator exhausted.")

# The iterator is now exhausted
print(f"Trying list() on exhausted iterator: {list(my_iterator)}")

# For loop handles this automatically
print("\nUsing a for loop (on a new iterator):")
for item in my_list: # Implicitly calls iter() and next()
    print(f"  Item: {item}")

Is list iterable? True
Type of iterator: <class 'list_iterator'>
Does iterator have __iter__? True
Does iterator have __next__? True

Manual iteration:
  Item 1: a
  Item 2: b
  Item 3: c
  Caught StopIteration - iterator exhausted.
Trying list() on exhausted iterator: []

Using a for loop (on a new iterator):
  Item: a
  Item: b
  Item: c


## 3. Generator Functions: `yield`

The easiest way to create an iterator is using a generator function. This looks like a normal function but uses the `yield` keyword to return values one at a time.

*   When a generator function is called, it returns a **generator object** (which is an iterator) without executing the function body yet.
*   When `next()` is called on the generator object for the first time, execution starts and runs until the first `yield` statement is encountered.
*   The value specified after `yield` is returned by `next()`.
*   The function's state (local variables, instruction pointer) is **paused** at the `yield`.
*   On subsequent calls to `next()`, execution resumes *immediately after* the last `yield` statement, continuing until the next `yield` or the function ends.
*   If the function finishes without hitting another `yield`, `StopIteration` is raised automatically.

In [2]:
from typing import Generator, Any

def simple_generator(n: int) -> Generator[int, None, None]: # Type hint for generator
    """Yields numbers from 0 up to (but not including) n."""
    print("-> Generator function called, returning generator object.")
    i = 0
    while i < n:
        print(f"--> Inside generator: before yielding {i}")
        yield i
        print(f"<-- Inside generator: after yielding {i}")
        i += 1
    print("-> Generator function finished.")
    # No explicit StopIteration needed

# Call the function - it returns the generator object immediately
gen_obj = simple_generator(3)
print(f"Type of gen_obj: {type(gen_obj)}")

# Manually iterate using next()
print("\n--- Manual iteration with next() ---")
try:
    val1 = next(gen_obj)
    print(f"Received from next(): {val1}")
    
    val2 = next(gen_obj)
    print(f"Received from next(): {val2}")
    
    val3 = next(gen_obj)
    print(f"Received from next(): {val3}")
    
    val4 = next(gen_obj) # Will raise StopIteration
    print(f"Received from next(): {val4}")
except StopIteration:
    print("Caught StopIteration.")

# Using a for loop (more typical)
print("\n--- Iteration with for loop (on a new generator object) ---")
for value in simple_generator(2):
    print(f"Received in for loop: {value}")

Type of gen_obj: <class 'generator'>

--- Manual iteration with next() ---
-> Generator function called, returning generator object.
--> Inside generator: before yielding 0
Received from next(): 0
<-- Inside generator: after yielding 0
--> Inside generator: before yielding 1
Received from next(): 1
<-- Inside generator: after yielding 1
--> Inside generator: before yielding 2
Received from next(): 2
<-- Inside generator: after yielding 2
-> Generator function finished.
Caught StopIteration.

--- Iteration with for loop (on a new generator object) ---
-> Generator function called, returning generator object.
--> Inside generator: before yielding 0
Received in for loop: 0
<-- Inside generator: after yielding 0
--> Inside generator: before yielding 1
Received in for loop: 1
<-- Inside generator: after yielding 1
-> Generator function finished.


### 3.1 Memory Efficiency Demonstration

In [3]:
import sys

# Function returning a potentially large list
def create_large_list(n: int) -> list:
    print(f"Creating list with {n} elements...")
    return list(range(n))

# Generator function yielding elements
def generate_large_sequence(n: int) -> Generator[int, None, None]:
    print(f"Starting generator for {n} elements...")
    for i in range(n):
        yield i

num_elements = 1_000_000 # One million

# --- Using List (High Memory) --- 
print("--- List Approach ---")
large_list = create_large_list(num_elements)
print(f"Size of list object: {sys.getsizeof(large_list):,} bytes")
# Processing the list (example: summing first 10)
list_sum_first_10 = sum(large_list[:10])
print(f"Sum of first 10 from list: {list_sum_first_10}")
# Keep large_list in memory for comparison

# --- Using Generator (Low Memory) --- 
print("\n--- Generator Approach ---")
large_gen = generate_large_sequence(num_elements)
print(f"Size of generator object: {sys.getsizeof(large_gen):,} bytes") # Size is constant!
# Processing the generator (example: summing first 10)
gen_sum_first_10 = 0
for i, val in enumerate(large_gen):
    if i >= 10:
        break # Stop consuming after 10 items
    gen_sum_first_10 += val
print(f"Sum of first 10 from generator: {gen_sum_first_10}")

# The generator is partially consumed, but the full sequence was never in memory
# If we now sum the *rest* of the generator:
# total_sum_gen = gen_sum_first_10 + sum(large_gen) # sum() consumes the rest
# print(f"Total sum from generator: {total_sum_gen}")

del large_list # Free memory from the list example
del large_gen  # Free generator object

--- List Approach ---
Creating list with 1000000 elements...
Size of list object: 8,000,056 bytes
Sum of first 10 from list: 45

--- Generator Approach ---
Size of generator object: 216 bytes
Starting generator for 1000000 elements...
Sum of first 10 from generator: 45


## 4. Generator Expressions

Similar to list comprehensions, but create generator objects instead of lists. Use parentheses `()` instead of square brackets `[]`.

**Syntax:** `(expression for item in iterable if condition)`

They provide a concise way to create simple generators, often replacing simple uses of generator functions or `map`/`filter`.

In [4]:
# List comprehension (creates full list)
squares_list = [x*x for x in range(10)]
print(f"Squares list: {squares_list}")
print(f"Type: {type(squares_list)}")

# Generator expression (creates a generator object)
squares_gen = (x*x for x in range(10))
print(f"\nSquares generator object: {squares_gen}")
print(f"Type: {type(squares_gen)}")

# Iterate over the generator to get values
print(f"Values from squares generator: {list(squares_gen)}")

# Example with condition
even_squares_gen = (x*x for x in range(10) if x % 2 == 0)
print(f"\nEven squares generator: {list(even_squares_gen)}")

# Can be passed directly to functions that accept iterables
total_sum_of_squares = sum(x*x for x in range(10))
print(f"Sum of squares (using genexpr): {total_sum_of_squares}")

Squares list: [0, 1, 4, 9, 16, 25, 36, 49, 64, 81]
Type: <class 'list'>

Squares generator object: <generator object <genexpr> at 0x70196d414fb0>
Type: <class 'generator'>
Values from squares generator: [0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

Even squares generator: [0, 4, 16, 36, 64]
Sum of squares (using genexpr): 285


## 5. Advanced Generator Features

### 5.1 `yield from`: Delegating to Sub-Generators (Python 3.3+)

Allows a generator to delegate part of its operations to another generator (or any iterable), avoiding the need for an explicit inner loop.

**Syntax:** `yield from other_iterable`

In [5]:
from typing import Iterable, TypeVar, Generator

T = TypeVar('T')

def sub_generator_letters() -> Generator[str, None, None]:
    yield 'A'
    yield 'B'

def sub_generator_numbers() -> Generator[int, None, None]:
    yield 1
    yield 2

def main_generator() -> Generator[Any, None, None]:
    print("Main generator starting")
    # Without yield from:
    # for letter in sub_generator_letters():
    #     yield letter
    # for number in sub_generator_numbers():
    #     yield number
    
    # With yield from (cleaner):
    yield from sub_generator_letters() 
    yield "---" # Can yield other values in between
    yield from sub_generator_numbers()
    yield from range(3) # Can yield from any iterable
    print("Main generator finished")

print("--- Using yield from ---")
combined_gen = main_generator()
print(f"Combined results: {list(combined_gen)}")

--- Using yield from ---
Main generator starting
Main generator finished
Combined results: ['A', 'B', '---', 1, 2, 0, 1, 2]


### 5.2 Sending Values into Generators (`.send()`)

Generators can also *receive* values using the `send()` method. The `yield` expression itself evaluates to the value sent.
This turns generators into a form of **coroutine** (though Python's `async/await` is the modern way for explicit asynchronous programming).

In [6]:
from typing import Generator, Optional, Any

def receiver_generator() -> Generator[str, str, None]: # Yields str, Receives str, Returns None
    print("Receiver starting...")
    received_value = None
    while True:
        try:
            # Yield a value, and pause here to potentially receive a sent value
            received_value = yield f"Ready to receive (last was: {received_value})"
            print(f"--> Receiver got: {received_value!r}")
            if received_value == 'stop':
                print("Receiver stopping.")
                break
        except GeneratorExit:
            print("Receiver closing gracefully.")
            raise # Must re-raise GeneratorExit
        except Exception as e:
            print(f"Receiver caught exception: {e}")

recv_gen = receiver_generator()

# Must call next() first to advance to the first yield
initial_yield = next(recv_gen)
print(f"Initial yield from receiver: {initial_yield!r}")

# Send values into the generator
yield1 = recv_gen.send("Hello")
print(f"Yield after send('Hello'): {yield1!r}")

yield2 = recv_gen.send(123)
print(f"Yield after send(123): {yield2!r}")

# Stop the generator by sending a specific value
try:
    recv_gen.send('stop')
except StopIteration:
    print("Receiver finished after 'stop'.")

# Closing a generator
recv_gen_close = receiver_generator()
next(recv_gen_close) # Start it
recv_gen_close.close() # Raises GeneratorExit inside the generator

Receiver starting...
Initial yield from receiver: 'Ready to receive (last was: None)'
--> Receiver got: 'Hello'
Yield after send('Hello'): 'Ready to receive (last was: Hello)'
--> Receiver got: 123
Yield after send(123): 'Ready to receive (last was: 123)'
--> Receiver got: 'stop'
Receiver stopping.
Receiver finished after 'stop'.
Receiver starting...
Receiver closing gracefully.


### 5.3 Asynchronous Generators (`async def` with `yield` - Python 3.6+)

Used with `asyncio` to create iterators that can pause during `await` calls, allowing other asynchronous tasks to run. Consumed using `async for`.

**(Note:** Requires an `asyncio` event loop to run. We won't run a full example here, but show the structure.)

In [7]:
import asyncio
from typing import AsyncGenerator

async def async_data_fetcher(urls: list) -> AsyncGenerator[str, None]:
    """Example async generator (conceptual)."""
    print("Starting async fetcher...")
    for url in urls:
        print(f"  Fetching {url}...")
        # Simulate an async network request
        await asyncio.sleep(0.5) 
        result = f"Data from {url}"
        yield result # Yield data as it becomes available
    print("Async fetcher finished.")

async def main_async():
    urls_to_fetch = ["http://example.com/1", "http://example.com/2", "http://example.com/3"]
    
    print("Iterating over async generator:")
    # Use 'async for' to consume an async generator
    async for data in async_data_fetcher(urls_to_fetch):
        print(f"  Received async data: {data}")

# To run this, you would typically use:
# asyncio.run(main_async())
print("(Async example structure shown, not executed in this notebook)")

(Async example structure shown, not executed in this notebook)


## 6. The `itertools` Module: Iteration Power Tools

This module (covered briefly before) provides building blocks for creating complex iterator logic. It's highly relevant to functional programming and generator usage.

**Examples Revisited:** `chain`, `islice`, `combinations`, `permutations`, `cycle`, `repeat`, `count`, `accumulate`, `groupby`, `takewhile`, `dropwhile`, `zip_longest`, etc.

In [8]:
import itertools

# Example: Find runs of consecutive numbers
data = [1, 2, 3, 5, 6, 8, 9, 10]

# groupby groups consecutive items based on a key function
# Here, key = value - index. This key is constant for consecutive numbers.
print("--- Grouping Consecutive Numbers --- ")
for k, g in itertools.groupby(enumerate(data), lambda item: item[1] - item[0]):
    run = [item[1] for item in g] # Extract values from the group
    print(f"  Run found: {run}")

# Example: Create pairs from two lists, padding shorter one
keys = ['a', 'b', 'c']
values = [1, 2]
paired = itertools.zip_longest(keys, values, fillvalue=None)
print("\n--- zip_longest Example ---")
print(list(paired))

--- Grouping Consecutive Numbers --- 
  Run found: [1, 2, 3]
  Run found: [5, 6]
  Run found: [8, 9, 10]

--- zip_longest Example ---
[('a', 1), ('b', 2), ('c', None)]


## 7. Best Practices & Enterprise Considerations

1.  **Use Generators for Large/Infinite Sequences:** Prefer generators or generator expressions over lists when dealing with potentially large amounts of data to conserve memory.
2.  **Favor Generator Expressions for Simplicity:** For straightforward map/filter operations without complex logic, generator expressions are often the most concise and Pythonic.
3.  **Readability is Key:** While generators can be powerful, ensure the logic remains understandable. A slightly more verbose `for` loop might be better than an overly complex generator expression.
4.  **Iterators are Single-Pass:** Remember that standard iterators (including generators) can only be consumed once. If you need to iterate multiple times, either convert to a list/tuple first (if memory allows) or regenerate the iterator.
5.  **Chaining:** Build complex data processing pipelines by chaining generators together. This remains memory-efficient as data flows through the pipeline item by item.
6.  **Error Handling:** Include `try...except` blocks within your generator functions if individual items might cause errors during generation, allowing the generator to continue if appropriate.
7.  **Resource Management:** If your generator function opens resources (like files), ensure they are closed properly, potentially using a `try...finally` block or a context manager *inside* the generator function.
8.  **`yield from`:** Use `yield from` to simplify code when delegating iteration to another iterable within a generator.

## 8. Pitfalls and Common Interview Questions

**Common Pitfalls:**

*   **Treating Generators like Lists:** Trying to index (`gen[0]`) or get the length (`len(gen)`) of a generator directly (doesn't work without consuming it into a list).
*   **Forgetting Iterators are Single-Pass:** Trying to loop over the same generator object multiple times.
*   **Memory Issues with Intermediate Lists:** Converting large generators to lists unnecessarily (e.g., `list(map(...))` when the result will just be iterated over anyway).
*   **Unintended Side Effects in Generators:** Performing actions with side effects within a generator might lead to unexpected behavior due to lazy evaluation (the side effect only happens when that `yield` is reached).
*   **Blocking Generators:** Performing long-running synchronous operations inside a generator can block consumption (especially relevant in async contexts).

**Common Interview Questions:**

1.  What is the difference between an iterable and an iterator in Python?
2.  How does a generator function differ from a regular function? What does `yield` do?
3.  What are the main advantages of using generators?
4.  What is a generator expression? How does its syntax differ from a list comprehension?
5.  Can you iterate over a generator multiple times? Why or why not?
6.  Give an example of where using a generator would be more memory-efficient than using a list.
7.  What does `yield from` do?
8.  (Advanced) Can you send values *into* a generator? How? (`send()`)
9.  How do generators relate to lazy evaluation?

## 9. Challenge: Streaming CSV Reader

**Goal:** Create a generator function that reads a potentially large CSV file row by row, yielding each row as a dictionary, without loading the entire file into memory.

**Tasks:**

1.  **Create Sample CSV:** Create a sample CSV file (`sensor_data.csv`) with headers and a few rows (or many rows to simulate size):
    ```csv
    Timestamp,SensorID,Temperature,Humidity
    2023-11-01T10:00:00Z,SensorA,22.5,45.1
    2023-11-01T10:01:00Z,SensorB,21.8,46.5
    2023-11-01T10:02:00Z,SensorA,22.6,45.2
    INVALID LINE
    2023-11-01T10:03:00Z,SensorC,23.1,44.9
    2023-11-01T10:04:00Z,SensorB,21.9,46.8
    ```
2.  **Write Generator Function:** Create a function `read_csv_as_dicts(filepath: Path) -> Generator[Dict[str, str], None, None]`:
    *   It should take the file path (`pathlib.Path` object) as input.
    *   Open the file using `with open(...)`.
    *   Read the header row separately.
    *   For each subsequent row:
        *   Split the row into fields (handle potential errors like incorrect number of fields).
        *   Create a dictionary mapping header names to field values for that row.
        *   `yield` the dictionary.
    *   Include basic error handling (e.g., `FileNotFoundError`, skipping lines with wrong field count with a log message).
3.  **Test:**
    *   Call your generator function.
    *   Iterate through the resulting generator object using a `for` loop.
    *   Print each dictionary (row) received.
    *   (Optional) Process only the first few rows to demonstrate lazy evaluation.

**(Bonus):** Modify the generator to optionally attempt type conversion (e.g., float for Temperature/Humidity) within a `try...except ValueError` block.

In [9]:
# --- Solution Space for Challenge ---
import csv # Can use csv module for more robust parsing!
from pathlib import Path
from typing import Generator, Dict, List, Optional, Union
import logging

logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s', force=True)

# 1. Create Sample CSV
csv_content = """
Timestamp,SensorID,Temperature,Humidity
2023-11-01T10:00:00Z,SensorA,22.5,45.1
2023-11-01T10:01:00Z,SensorB,21.8,46.5
2023-11-01T10:02:00Z,SensorA,22.6,45.2
INVALID LINE, only one field
2023-11-01T10:03:00Z,SensorC,23.1,44.9
2023-11-01T10:04:00Z,SensorB,21.9,46.8,extra_field
"""
csv_filepath = Path("sensor_data.csv")
try:
    csv_filepath.write_text(csv_content.strip(), encoding='utf-8')
    logging.info(f"Created sample CSV file: {csv_filepath}")
except IOError as e:
    logging.error(f"Failed to write CSV file: {e}")
    exit()

# 2. Write Generator Function (using csv module for robustness)
def read_csv_as_dicts(filepath: Path, delimiter: str = ',') -> Generator[Dict[str, str], None, None]:
    """Reads a CSV file line by line, yielding each row as a dictionary."""
    try:
        with filepath.open('r', encoding='utf-8', newline='') as f:
            # Use csv.reader for robust parsing
            reader = csv.reader(f, delimiter=delimiter)
            try:
                header = next(reader) # Read header row
                logging.info(f"CSV Headers: {header}")
                num_headers = len(header)
                
                for i, row_list in enumerate(reader, 2): # Start line count from 2
                    if len(row_list) != num_headers:
                        logging.warning(f"Line {i}: Skipping row. Expected {num_headers} fields, got {len(row_list)}. Row: {row_list}")
                        continue # Skip malformed row
                    
                    # Create dictionary using zip
                    row_dict = dict(zip(header, row_list))
                    yield row_dict # Yield the dictionary for this row
                    
            except StopIteration:
                logging.warning(f"CSV file '{filepath}' is empty or has only a header.")
            except csv.Error as csv_e:
                 logging.error(f"CSV parsing error in '{filepath}' near line {reader.line_num}: {csv_e}")
                 # Could choose to stop or continue here
                 
    except FileNotFoundError:
        logging.error(f"File not found: {filepath}")
        # Optionally raise an error here if needed
        # raise
    except IOError as e:
        logging.error(f"IOError reading file {filepath}: {e}")
    except Exception as e:
         logging.exception(f"Unexpected error reading CSV {filepath}: {e}")

# Bonus: Generator with Type Conversion
def read_csv_converted(filepath: Path, delimiter: str = ',') -> Generator[Dict[str, Any], None, None]:
     for row_dict in read_csv_as_dicts(filepath, delimiter):
         converted_row = row_dict.copy() # Work on a copy
         try:
            # Attempt conversions - add more as needed
            if 'Temperature' in converted_row:
                converted_row['Temperature'] = float(converted_row['Temperature'])
            if 'Humidity' in converted_row:
                 converted_row['Humidity'] = float(converted_row['Humidity'])
            # Could add datetime parsing for 'Timestamp' here too
            yield converted_row
         except (ValueError, TypeError) as conv_e:
             logging.warning(f"Could not convert types for row: {row_dict}. Error: {conv_e}")
             # Decide whether to yield original row, None, or skip
             # yield row_dict # Yield original if conversion fails
             continue # Skip row if conversion fails

# 3. Test
print("\n--- Testing CSV Generator (Raw Strings) ---")
row_count = 0
csv_gen = read_csv_as_dicts(csv_filepath)
for row in csv_gen:
    row_count += 1
    print(f"  Row {row_count}: {row}")
    if row_count == 3:
         print("  (Stopping early to demonstrate lazy evaluation)")
         # break # Uncomment to stop early

print("\n--- Testing CSV Generator (with Type Conversion) ---")
row_count_conv = 0
csv_gen_conv = read_csv_converted(csv_filepath)
for row_conv in csv_gen_conv:
     row_count_conv += 1
     print(f"  Conv Row {row_count_conv}: {row_conv} (Temp Type: {type(row_conv.get('Temperature'))})")

INFO: Created sample CSV file: sensor_data.csv
INFO: CSV Headers: ['Timestamp', 'SensorID', 'Temperature', 'Humidity']
INFO: CSV Headers: ['Timestamp', 'SensorID', 'Temperature', 'Humidity']



--- Testing CSV Generator (Raw Strings) ---
  Row 1: {'Timestamp': '2023-11-01T10:00:00Z', 'SensorID': 'SensorA', 'Temperature': '22.5', 'Humidity': '45.1'}
  Row 2: {'Timestamp': '2023-11-01T10:01:00Z', 'SensorID': 'SensorB', 'Temperature': '21.8', 'Humidity': '46.5'}
  Row 3: {'Timestamp': '2023-11-01T10:02:00Z', 'SensorID': 'SensorA', 'Temperature': '22.6', 'Humidity': '45.2'}
  (Stopping early to demonstrate lazy evaluation)
  Row 4: {'Timestamp': '2023-11-01T10:03:00Z', 'SensorID': 'SensorC', 'Temperature': '23.1', 'Humidity': '44.9'}

--- Testing CSV Generator (with Type Conversion) ---
  Conv Row 1: {'Timestamp': '2023-11-01T10:00:00Z', 'SensorID': 'SensorA', 'Temperature': 22.5, 'Humidity': 45.1} (Temp Type: <class 'float'>)
  Conv Row 2: {'Timestamp': '2023-11-01T10:01:00Z', 'SensorID': 'SensorB', 'Temperature': 21.8, 'Humidity': 46.5} (Temp Type: <class 'float'>)
  Conv Row 3: {'Timestamp': '2023-11-01T10:02:00Z', 'SensorID': 'SensorA', 'Temperature': 22.6, 'Humidity': 45.2}

## 10. Conclusion

Generators are a cornerstone of efficient and elegant Python programming, particularly when dealing with iteration and data processing. By embracing lazy evaluation through generator functions (`yield`) and generator expressions (`(...)`), you can significantly reduce memory consumption, handle potentially infinite sequences, and build clean, composable data pipelines.

Understanding the underlying iterator protocol provides a solid foundation, while tools like `yield from` and the `itertools` module offer further power and conciseness. When combined with functional concepts, generators enable a highly effective style for tackling many common programming challenges in Python.